In [2]:
import math
import random
import sys

sys.path.append('../')

from utilities.utils import *
from addressPoint import AddressPoint

from geopy.geocoders import Nominatim
import folium

In [3]:
target_address = '9 Boulevard Decouz, 74000 Annecy France'
central_point = AddressPoint(target_address)

In [4]:
def simulate_points(num_points, center_point, max_radius):
    geolocator = Nominatim(user_agent="covoiturage")
    min_long = 6
    max_long = 7
    min_lat =45
    max_lat = 46
    long_lat_points={}
    random.seed(42316)
    countall = 0

    while len(long_lat_points) < num_points:
        keep = True
        rho, theta = random.uniform(0, max_radius), random.uniform(-math.pi, math.pi)
        x_in = rho * math.cos(theta)
        y_in = rho * math.sin(theta)
        lat_in, long_in = latitude_longitude_from_cartesians(x_in, y_in, center_point, center_point.latitude)
        strAddress_out = geolocator.reverse(query=(lat_in, long_in))
        if strAddress_out is not None:
            strAddressList = str(strAddress_out).split(',')
            char1 = strAddressList[0][0]
            strAddress_simple = '{}, {}'.format(strAddressList[0], strAddressList[-2])
            if not(char1.isnumeric()):
                strAddress_simple = '1 {}'.format(strAddress_simple)
            try:
                lat, long = get_latitude_longitude(strAddress_simple)
                if (lat, long) == (None, None):
                    keep = False
                else:
                    if long<min_long or long>max_long or lat<min_lat or lat>max_lat:
                        keep = False
            except:
                print('error for address: {}'.format(strAddress_simple))
                keep = False
            if keep:
                long_lat_points[strAddress_simple] = (lat, long)                
        countall+=1
        print('global count: {}, valid count: {}, @: {}'.format(countall, len(long_lat_points), strAddress_simple))
    return long_lat_points

In [ ]:
address_points = simulate_points(100, central_point, 10)

In [ ]:
full_map = folium.Map(location=[central_point.latitude, central_point.longitude], zoom_start=12)
folium.Marker([central_point.latitude, central_point.longitude]).add_to(full_map)
for p in address_points.values():
    folium.Circle(radius = 10, location=[p[0], p[1]], color='red', fill=True).add_to(full_map)
full_map

In [ ]:
with open('./data/addresses_test.csv', 'w') as f:
    f.write('first_name;last_name;address\n')
    for k in address_points:
        f.write('x;x;{}\n'.format(k))